In [34]:
# ---------- 1. Imports ----------
from typing import TypedDict, Optional
from langgraph.graph import StateGraph
from langchain_core.runnables import RunnableLambda
from langchain.agents import Tool, initialize_agent
from langchain.agents.agent_types import AgentType
from langchain_groq import ChatGroq

In [44]:
# ---------- 2. State schema ----------
class AgentState(TypedDict):
    input: str                 # user question
    response: Optional[str]    # final answer

In [45]:
# ---------- 3. Tools ----------
tools = [
    Tool(name="get_time",
         func=lambda _: "12:00 PM",
         description="Return the current time"),
    Tool(name="get_weather",
         func=lambda _: "Sunny",
         description="Return today's weather"),
]

In [ ]:
# ---------- 4. LLM + Agent ----------
llm = ChatGroq(model="llama-3.3-70b-versatile", api_key="GROQ_API_KEY")

agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=False
)

In [47]:
# ---------- 5. Node functions ----------
def agent_node(state: AgentState) -> AgentState:
    """Ask the LLM what to do next (it chooses a tool or gives a direct answer)."""
    result = agent.invoke({"input": state["input"]})
    # result["output"] holds the agent’s spoken answer
    state["agent_output"] = result["output"]
    return state

def branch_key(state: AgentState) -> str:
    """Decide which tool node to visit based on the user's input."""
    text = state["input"].lower()
    return "get_time" if "time" in text else "get_weather"

def get_time_node(state: AgentState) -> AgentState:
    state["response"] = "🕒 " + tools[0].func(None)
    return state

def get_weather_node(state: AgentState) -> AgentState:
    state["response"] = "🌤️ " + tools[1].func(None)
    return state

def end_node(state: AgentState) -> AgentState:
    return state

In [48]:
# ---------- 6. Build the LangGraph ----------
builder = StateGraph(state_schema=AgentState)

builder.add_node("agent",       RunnableLambda(agent_node))
builder.add_node("get_time",    RunnableLambda(get_time_node))
builder.add_node("get_weather", RunnableLambda(get_weather_node))
builder.add_node("end",         RunnableLambda(end_node))

builder.set_entry_point("agent")

In [49]:
# conditional branch straight from agent → (tool)
builder.add_conditional_edges(
    "agent",
    RunnableLambda(branch_key),      # how to pick the path
    {
        "get_time":    "get_time",
        "get_weather": "get_weather",
    },
)

In [50]:
# each tool → end
builder.add_edge("get_time",    "end")
builder.add_edge("get_weather", "end")

graph = builder.compile()

In [51]:
# ---------- 7. Run ----------
if __name__ == "__main__":
    for question in ["What's the weather in pakistan?", "culture in pakistan"]:
        result = graph.invoke({"input": question})
        print(f"Q: {question}\nA: {result['response']}\n")


Q: What's the weather in pakistan?
A: 🌤️ Sunny

Q: culture in pakistan
A: 🌤️ Sunny

